In [2]:
import torch

In [3]:
print(torch.__version__)

2.6.0


In [4]:
print(torch.backends.mps.is_available())

True


In [5]:
print(torch.rand(2,2))

tensor([[0.5621, 0.1849],
        [0.5342, 0.7369]])


## Tensors
A tensor is both a container for numbersas well as a set of rules that define transformations between tensors that produce new tensors. 

Easiest to think about tensors as multidimensional arrays

Every tensor has a rank that corresponds to its dimensional space

In [6]:
x = torch.tensor([[0,0,1], [1,1,1], [0,0,0]])
x

tensor([[0, 0, 1],
        [1, 1, 1],
        [0, 0, 0]])

we can change an element in a tensor by using standard python indexing:

In [8]:
x[0][0] = 5
x

tensor([[5, 0, 1],
        [1, 1, 1],
        [0, 0, 0]])

special functions can generate particular types of tensors, such as `ones` and `zeroes`

In [9]:
torch.zeros(2,2)

tensor([[0., 0.],
        [0., 0.]])

In [14]:
torch.ones(1, 2) + torch.ones(1,2)

tensor([[2., 2.]])

and if you have a tensor of rank 0, you can pull  out the value with `item()`:

In [15]:
torch.rand(1).item()

0.29672127962112427

tensors can live in the cpu and on the gpu and be copied between devices by using the `to()` function

In [18]:
cpu_tensor = torch.rand(2)
print(cpu_tensor.device)
mps_tensor = cpu_tensor.to('mps')
print(mps_tensor.device)

cpu
mps:0


## Tensor Operations

1. Find the maximum item in a tensor as well as the index that contains the maximum value, done with the max() and argmax() functions. We can also use item() to extract a standard python value from a 1D tensor.

In [20]:
print(torch.rand(2,2).max())
print(torch.rand(2,2).max().item())

tensor(0.9931)
0.6906684041023254


Sometimes we want to change the type of a tensor - such as from a long tensor to a float tensor. We can do this with `to():`

In [23]:
long_tensor = torch.tensor([[0,0,1], [1,1,1], [0,0,0]])
print(long_tensor.type())

float_tensor = torch.tensor([[0,0,1], [1,1,1], [0,0,0]]).to(dtype=torch.float32)
print(float_tensor.type())

torch.LongTensor
torch.FloatTensor


If you want to save memory you can use an *in place* funcction:

Look to see if an in place function is defined, which should be the same name as the original function but with an appended underscore(_):

In [24]:
random_tensor = torch.rand(2,2)
random_tensor.log2()

tensor([[-0.5836, -0.0632],
        [-1.5603, -2.7241]])

In [25]:
random_tensor.log2_()

tensor([[-0.5836, -0.0632],
        [-1.5603, -2.7241]])

another common operation is reshaping a tensor. This can occur because your NN layer may require a slightly different input shape than that you currently have to feed into it:

In [28]:
flat_tensor = torch.rand(784)
print(flat_tensor.shape)

viewed_tensor = flat_tensor.view(1, 28, 28)
print(viewed_tensor.shape)

reshaped_tensor = flat_tensor.reshape(1, 28, 28)
print(reshaped_tensor.shape)

torch.Size([784])
torch.Size([1, 28, 28])
torch.Size([1, 28, 28])


note that the reshaped tensor has to ahve he same number of total elements as the original, if you try `flat_tensor.reshape(3, 28, 28)`, you'll see an error.

#### view vs reshape
- view operatesas a view of the original tensor, so if the underlying data is changed, teh view will change too. 
- view can throw errors if the required block is not *contiguous* (it doesn't share the same block of memory it would occupy if a new tensor of the required shape was created from scratch). 
- IF this happens, you have to call `tensor.contiguous()` before you can use `view()`. 

However, reshape does it all behind the scenes.

### Rearranging the dimensions of a tensor
You will likely come across this with images, which are often stored as `[height, width, channel]` tensors, but pytorch prefers to deal with these in a `[channel, height, width]`. 

You can use `permute` to deal with these in a fairly straightforward manner:

In [29]:
hwc_tensor = torch.rand(640, 480, 3)
chw_tensor = hwc_tensor.permute(2, 0, 1)
print(chw_tensor.shape)

torch.Size([3, 640, 480])


## Tensor Broadcasting
Broadcasting allows you to perform operations between a tensor and a smaller tensor. You can boradcast acrosstwo tensors if, starting backward from their trailing dimensions:
- the two dimensions are equal
- One of the dimensions is 1.

In our use of broadcasting, it works because 1 has a dimensions of 1, and there are no other dimensions, the 1 can be expanded to cover the other tensor.

If we tried to add a `[2,2]` tensor to a `[3,3]` tensor, we'd get an error message, but we could add a `[1,3]` tensor to the `[3,3]` tensor without any trouble. 